<a href="https://colab.research.google.com/github/RajSinha77/DL-Reg-implementation-/blob/main/DLReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install transformers datasets

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from datasets import load_dataset
from PIL import Image

# Load Tiny-ImageNet dataset
dataset = load_dataset('zh-plus/tiny-imagenet')

# Check available splits
print(dataset)

# Define transformations for the training and validation sets
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2770, 0.2691, 0.2821]),
])

# Function to ensure image has 3 channels
def ensure_three_channels(img):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img

# Prepare the dataset
def preprocess_dataset(dataset, split):
    dataset = dataset[split]
    images = []
    labels = []
    for i in range(len(dataset)):
        img = dataset[i]['image']
        img = ensure_three_channels(img)
        img = transform(img)
        label = dataset[i]['label']
        images.append(img)
        labels.append(label)
    return torch.stack(images), torch.tensor(labels)

train_images, train_labels = preprocess_dataset(dataset, 'train')
val_images, val_labels = preprocess_dataset(dataset, 'test')

train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
val_dataset = torch.utils.data.TensorDataset(val_images, val_labels)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# Define the model with DL-Reg regularization
class DLRegNet(nn.Module):
    def __init__(self, base_model):
        super(DLRegNet, self).__init__()
        self.base_model = base_model
        self.regularization_factor = 1e-4

    def forward(self, x):
        return self.base_model(x)

    def compute_dl_reg_loss(self, x, output):
        x_flat = x.view(x.size(0), -1)
        output_flat = output.view(output.size(0), -1)
        Z = torch.linalg.lstsq(x_flat, output_flat).solution
        reg_loss = torch.norm(x_flat @ Z - output_flat, p='fro') ** 2
        return self.regularization_factor * reg_loss

# Load pre-trained model
base_model = models.resnet18(pretrained=True)
num_features = base_model.fc.in_features
base_model.fc = nn.Linear(num_features, 200)  # Tiny-ImageNet has 200 classes

model = DLRegNet(base_model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            reg_loss = model.compute_dl_reg_loss(inputs, outputs)
            total_loss = loss + reg_loss
            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# Move model to GPU
model.cuda()

# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer, num_epochs=25)
evaluate_model(model, val_loader)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
    valid: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})


In [3]:
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
    valid: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})
